In [1]:
import numpy as np
from PIL import Image
from glob import glob # manipulação de lista de nomes de arquivos
from skimage.measure import regionprops # propriedades de cada segmento
import matplotlib.pyplot as plt

import seaborn as sns # gráficos
import pandas as pd # manipulação de dataframes (sheets)

In [2]:
# Célula com funções para investigar se um segmento é misto, obter HoR e obter a classe majoritária de um segmento

# 0 - Fundo
# 1 - Desmatamento recente
# 2 - Floresta
# 3 - Ñ analisado

def is_mixed(segment):
    # flattening segment
    segment = segment.flatten()


    NFP = np.count_nonzero(segment == 2) # se floresta
    NP = np.count_nonzero(segment) # desconsiderando o fundo (np.zeros)
    NNP = NP - NFP # Calcula os pixels de não-fundo

    if NFP != 0 and NNP != 0: # se em um segmento tiver pixels de desmata e floresta, ele é misto
        return True
    
    return False
    

def get_hor(segment):
    
    # flattening segment (transformando a matriz 'segment' em um array unidimensional)
    segment = segment.flatten()

    # Number Foreground Pixels: conta o número de pixels no vetor segment que são diferentes de zero. Especificamente, os que têm o valor 2
    NFP = np.count_nonzero(segment == 2)

    # conta o número total de pixels no vetor 'segment' que são diferentes de zero. Isso representa o número total de pixels
    NP = np.count_nonzero(segment) # desconsiderando o fundo
    
    # Número de pixels de não-fundo: subtração da quantidade de pixels total de não fundo pela quantidade de pixels iguais a 170
    NNP = NP - NFP

    HoR = max([NFP, NNP]) / NP

    return HoR


def get_major_class(segment):
    if np.argmax(np.bincount(segment.flatten())) == 2:
        return "forest"
    elif np.argmax(np.bincount(segment.flatten())) == 1:
        return "non forest"
    elif np.argmax(np.bincount(segment.flatten())) == 3:
        return "not analyzed"
    else:
        return np.argmax(np.bincount(segment.flatten()))
    
def calculate_statistics(segmentation):
    unique_classes, counts = np.unique(segmentation, return_counts=True)
    std_dev = np.std(counts)
    mean = np.mean(counts)
    coeff = round(std_dev/mean, 4)

    return std_dev, mean, coeff

In [6]:
# Construir as listas de arquivos a processar
base = 'data/'
slic_path = base + 'saved_slic/'
truth_path = base + 'saved_truth/'
file_list = [
    'ae03_sentinel_rgb.npy',
    'sentinel_rgb_mask.npy',
    'AREA3_c_10000_hls_mask_corrigido.npy', 
    'AREA3_c_10000_rw_mask_corrigido.npy', 
    'AREA3_c_10000_snll_mask_corrigido.npy', 
    'AREA3_c_10000_wishart_mask_corrigido.npy']
sentinel_truth = 'ae03_truth.npy'
sentinel_big_truth = 'A3_big.npy'

parameters = {
    0: {
        'name': 'Sentinel-2',
        'truth': truth_path + sentinel_big_truth,
        'slic': slic_path + 'ae03_sentinel_rgb.npy'
    },
    1: {
        'name': 'Sentinel-2 masked',
        'truth': truth_path + sentinel_big_truth,
        'slic': slic_path + 'sentinel_rgb_mask.npy'
    },
    2: {
        'name': 'Wishart mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_wishart_mask_corrigido.npy'
    },
    3: {
        'name': 'RW mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_rw_mask_corrigido.npy'
    },
    4: {
        'name': 'SNLL mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_snll_mask_corrigido.npy'
    },
    5: {
        'name': 'HLT mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_hls_mask_corrigido.npy'
    }
}

In [8]:
global_segment_array = [] # segs
global_perfect_segments = [] # perfect segs

for idx in parameters.keys(): # para cada uma das imagens (ou slics)
    total_segs = 0 
    good_segs = 0
    bad_by_hor = 0
    bad_by_size = 0
    n_perfect_segs = 0 # Quando HoR = 1
    truth = np.load(parameters[idx].get('truth')) # Carrega a verdade da idx-ésima imagem (truth_x01.npy, por exemplo)
    slic = np.load(parameters[idx].get('slic')) # Carrega o slic da idx-ésima imagem (slic_x01.npy, por exemplo)

    props = regionprops(slic) # Calcula as propriedades de cada segmento e atribui a 'props'
    segments_truth = [] # List com os segmentos da verdade
    segments_image = [] # List com os segmentos da imagem
    bad_segment_ids = [] # List com os ids dos segmentos ruins
    perfect_segs_truth = [] # List com os segmentos perfeitos da verdade

    sd, mean, coeff = calculate_statistics(slic)

    assert slic.shape[:2] == truth.shape[:2] # Checa se o slic corrente possui o mesmo shape (h, w) da verdade. Caso contrário, a execução é interrompida
    
    for i, prop in enumerate(props): # para cada segmento i do slic idx...
        
        total_segs += 1 # count...
        
        minr, minc, maxr, maxc = prop.bbox # minr: linha superior, minc: coluna mais à esquerda, maxr: linha inferior, maxc: coluna mais à direita

        # As linhas a seguir criam estruturas (arrays) vazias para os segmentos da verdade e da imagem
        segment_truth = np.zeros((maxr - minr, maxc - minc)) # delimita o tamanho do segmento da verdade
        coords = np.array(prop.coords) # pega as coordenadas de cada pixel

        for pixel in coords: # para cada coordenada de pixel dentro do i-ésimo segmento
            segment_truth[pixel[0] - minr, pixel[1] - minc] = truth[pixel[0], pixel[1]] # faz uma cópia, pixel a pixel do i-ésimo segmento da verdade

        hor = get_hor(segment_truth) # HoR da verdade
        classification = get_major_class(np.uint8(segment_truth))

        # se a quantidade de pixels no i-ésimo segmento na verdade > 70, HoR desse segmento > 0.7 e ele é floresta ou não floresta: adicione o 
        if (segment_truth.shape[0] * segment_truth.shape[1] > 70) and (get_hor(segment_truth) > 0.7) and (classification in ["forest", "non forest"]):
            segments_truth.append(np.uint8(segment_truth))             
            good_segs += 1

        # se HoR do segmento atual é < 0.7... 
        if (hor <= 0.7):
            bad_by_hor += 1
            bad_segment_ids.append(prop.label)

        # se o tamanho do segmento atual é <= 70 pixels ou for fundo
        if ((segment_truth.shape[0] * segment_truth.shape[1] <= 70) or (classification == 0)):
            bad_by_size += 1
            bad_segment_ids.append(prop.label)

        # se o segmento atual é perfeito (HoR = 1)
        if (hor == 1):
            perfect_segs_truth.append(segment_truth)
            n_perfect_segs += 1
    
    # fim da análise por segmento

    # Adiciona os segmentos da verdade e os segmentos pefeitos à lista
    global_segment_array.append(segments_truth)
    global_perfect_segments.append(perfect_segs_truth)

    # Identifica os ids dos segmentos ruins. Praticamente não há, quando N_SEGMENTS é próximo a 8000
    bad_segment_ids = set(bad_segment_ids) # removes duplicates
    
    # Itera sobre os ids dos segmetnos ruins. Quando o segmento for ruim, a ele é atribuído -1
    for bad_id in bad_segment_ids:
        slic = np.where(slic == bad_id, -1, slic)

    # print(f'{parameters[idx].get('name')}: total_segs = {total_segs} | good_segs = {good_segs} | good%: {round(good_segs/total_segs, 4)}')
    latex_string = f'& {total_segs} & {good_segs} & {round(good_segs/total_segs, 4)} & {mean} & {sd} & {coeff} '
    print(latex_string) 

(2990, 2928)
(2990, 2928)
$ 9573 $ 5065 & 0.5291 & 914.5220933876527 & 144.63165899767438 & 0.1581 
(2990, 2928)
(2990, 2928)
$ 9830 $ 9341 & 0.9503 & 890.5218187366494 & 36555.5970311447 & 41.0496 
(2132, 2093)
(2132, 2093, 1)
$ 1752 $ 947 & 0.5405 & 2544.0570125427594 & 44289.192064310744 & 17.4089 
(2132, 2093)
(2132, 2093, 1)
$ 1704 $ 974 & 0.5716 & 2615.636576787808 & 44910.353870284824 & 17.17 
(2132, 2093)
(2132, 2093, 1)
$ 454 $ 217 & 0.478 & 9785.692982456141 & 100418.28178554443 & 10.2617 
(2132, 2093)
(2132, 2093, 1)
$ 2570 $ 1511 & 0.5879 & 1734.9440124416797 & 36462.642963655264 & 21.0166 
